In [138]:
%load_ext autoreload
%autoreload 2
import feature_extract
import sys
sys.path.append("..")
import torch
import psql_methods as psql
import pickle
import alchemy_methods as alc
from tqdm import tqdm
import numpy as np
import image_utils as imgs
import opensea_methods as opse
import multiprocessing
import pandas as pd
import feature_utils as feat
import matplotlib.pyplot as plt
import counterfeit_utils as cfu
import cf_value as cfv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
df = cfv.build_ko_rankings()

In [140]:
df.query("Top_100=='boredapeyachtclub' and Alt=='mutant-ape-yacht-club'")

,Top_100,Alt,Distance,sale_price,volume,Top_100_volume,Alt_date,Top_100_date,Top_ratio
171012,boredapeyachtclub,mutant-ape-yacht-club,0.525634,10.809283,1.102118e+06,1.460340e+06,1.630220e+09,1.619075e+09,0.7547


In [141]:
no_ders,w_ders = cfv.split_ders(df)
cfv.filter_one_top(w_ders)

,Top_100,Alt,Distance,sale_price,volume,Top_100_volume,Alt_date,Top_100_date,Top_ratio
171012,boredapeyachtclub,mutant-ape-yacht-club,0.525634,10.809283,1.102118e+06,1.460340e+06,1.630220e+09,1.619075e+09,0.754700
918101,mutant-ape-yacht-club,otherdeed,0.991348,3.383648,6.161550e+05,1.102118e+06,1.651216e+09,1.630220e+09,0.559064
99823,azuki,beanzofficial,0.782008,1.542050,1.768015e+05,7.225442e+05,1.648710e+09,1.641974e+09,0.244693
238702,clonex,rtfkt-mnlth,1.348469,5.182558,5.724784e+04,4.153179e+05,1.644048e+09,1.639296e+09,0.137841
1037993,proof-moonbirds,moonbirds-oddities,0.740094,1.227377,4.243084e+04,3.447647e+05,1.653203e+09,1.650006e+09,0.123072
470339,doodles-official,the-dooplicator,0.932024,1.137168,2.194396e+04,3.009661e+05,1.652425e+09,1.634454e+09,0.072912
160036,bored-ape-kennel-club,mutant-ape-yacht-club,0.924762,10.809283,1.102118e+06,2.829692e+05,1.630220e+09,1.624000e+09,3.894835
1059073,pudgypenguins,lilpudgys,0.431035,0.421603,4.696544e+04,2.822362e+05,1.639901e+09,1.626937e+09,0.166405
121562,beanzofficial,azukielementals,0.798747,0.979463,3.164365e+04,1.768015e+05,1.687331e+09,1.648710e+09,0.178978
265474,cool-cats-nft,coolpetsnft,0.541048,0.967766,3.154547e+04,1.484292e+05,1.643616e+09,1.624777e+09,0.212529


In [3]:
pw_dists = pd.read_pickle('pw_dists_counterfeit.pkl')
with open("../Graph_predictions/dataset_stor/graph_dataset_4/total_stats.pkl",'rb') as f:
    total_stats = pickle.load(f)

In [ ]:
key_list = list(total_stats.keys())
key_list = [x for x in key_list if total_stats[x][0]]
print(len(key_list))
temp_stats = opse.pull_nft_stats(key_list,no_save=True)

12282


 84%|█████████████████████████████████████████████████████████████████████████████▌              | 10350/12282 [28:34<04:43,  6.80it/s]

In [65]:
temp_stats_tuples = [tuple(list(x.values())[:8]) for x in temp_stats]
command = "INSERT INTO collection_total_stats (volume,sales, avg_price,num_owners,market_cap,floor,symbol,slug) VALUES (%s, %s, %s, %s,%s,%s,%s,%s)"
psql.batch_insert(command,temp_stats_tuples)

In [4]:
new_df = pd.DataFrame({'Top_100':pw_dists['Top_100'],'Alt': pw_dists['Alt'], 'Distance': pw_dists['Euc_Distance'],'sale_price': pw_dists['Alt'].map(lambda x: total_stats.get(x,[None,None,None])[2])})

In [67]:
unique_alts = result_df['Alt'].unique()
alt_date_map = {}
for alt in unique_alts:
    try:
        alt_date_map[alt] = cfu.creation_sec_from_db(alt)
    except:
        continue
unique_top_100s = result_df['Top_100'].unique()
top_100_map = {}
for top_100 in unique_top_100s:
    try:
        top_100_map[top_100] = stats_df.query(f"Alt=='{top_100}'")['volume'].iloc[0]
    except:
        continue
top_100_date_map = {}
for top_100 in unique_top_100s:
    try:
        top_100_date_map[top_100] = cfu.creation_sec_from_db(top_100)
    except:
        continue

In [91]:
command = "SELECT * from collection_total_stats"
temp_stats_tuples = psql.execute_commands([command])
new_df = pd.DataFrame({'Top_100':pw_dists['Top_100'],'Alt': pw_dists['Alt'], 'Distance': pw_dists['Euc_Distance'],'sale_price': pw_dists['Alt'].map(lambda x: total_stats.get(x,[None,None,None])[2])})
columns = ['volume','sales', 'avg_price','num_owners','market_cap','floor','symbol','Alt']
stats_df = pd.DataFrame(temp_stats_tuples,columns=columns)
vol_merge_df = pd.merge(new_df, stats_df[['Alt', 'volume']], on='Alt', how='left')
vol_merge_df

,Top_100,Alt,Distance,sale_price,volume
0,0n1-force,-glowa-,1.313444,0.164321,17.911000
1,0n1-force,-jubilees-multiverse,0.906586,0.009974,0.229397
2,0n1-force,-phunks,1.074891,0.003422,6.585190
3,0n1-force,-subtraction-,1.301293,0.652680,467.418697
4,0n1-force,0-project,1.302781,0.047461,67.679873
...,...,...,...,...,...
1459812,zombieclub-token,zunkpets,0.972472,0.031104,27.055308
1459813,zombieclub-token,zunkz,0.942404,0.114872,2269.067503
1459814,zombieclub-token,zuphioh-editions,0.856914,0.137089,5.072300
1459815,zombieclub-token,zuttomamoru,1.246206,0.019719,7.118630


In [69]:
n_smallest = 50
def keep_knocks(group):
    try:
        slug = group['Top_100'].iloc[0]
        command = f"select num from objective_cf_num where slug='{slug}' and type='pfps'"
        rows = psql.execute_commands([command])
        n_smallest = rows[0][0]
        return group.nsmallest(n_smallest,'Distance')
    except:
        return

# Keep the rows with the 100 smallest 'Euc_Distance' for each 'Top_100'
result_df = vol_merge_df.groupby('Top_100', group_keys=False).apply(keep_knocks)
result_df

,Top_100,Alt,Distance,sale_price,volume
6568,0n1-force,neotokyopunkscollabs,0.473301,0.026584,4.688860
10276,0n1-force,vainego,0.477493,0.001253,0.072670
5809,0n1-force,mekazuki,0.495278,0.032092,85.878284
8132,0n1-force,roars-by-ntp,0.511114,0.031096,129.816441
9945,0n1-force,tokyo-brave-heroes,0.517561,0.076197,272.328085
...,...,...,...,...,...
1415620,world-of-women-nft,world-of-boys-official,0.491815,0.006631,1.027800
1413889,world-of-women-nft,stardustgeneration,0.502492,0.032177,35.201774
1415600,world-of-women-nft,womenrise,0.505239,0.354732,6077.810595
1406738,world-of-women-nft,chillennials,0.524818,0.020226,8.090251


In [70]:
result_df['Top_100_volume'] = result_df['Top_100'].map(top_100_map)
result_df['Alt_date'] = result_df['Alt'].map(alt_date_map)
result_df['Top_100_date'] = result_df['Top_100'].map(top_100_date_map)

In [83]:
result_df['Top_ratio'] = result_df['volume']/result_df['Top_100_volume']
result_filt = result_df.sort_values(by='Top_ratio',ascending=False).query('Alt_date>Top_100_date')
result_filt

/tmp/ipykernel_161413/3140600736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Top_ratio'] = result_df['volume']/result_df['Top_100_volume']


,Top_100,Alt,Distance,sale_price,volume,Top_100_volume,Alt_date,Top_100_date,Top_ratio
694134,kiwami-genesis,degods,0.533451,4.507947,150996.553229,15693.694790,1.680160e+09,1648105200,9.621479
803894,metahero-generative,degods,0.488252,4.507947,150996.553229,19955.058789,1.680160e+09,1630825200,7.566831
699295,kiwami-genesis,pxnghostdivision,0.591871,1.947150,46716.851823,15693.694790,1.651734e+09,1648105200,2.976791
339038,cryptodickbutts-s3,thepotatoz,0.406820,1.427269,49694.345372,16938.398023,1.658387e+09,1627455600,2.933828
335525,cryptodickbutts-s3,moonbirds-oddities,0.444656,1.227377,42430.836151,16938.398023,1.653203e+09,1627455600,2.505009
...,...,...,...,...,...,...,...,...,...
1365713,veefriends-series-2,igocnp,0.552781,NaN,NaN,34475.032761,1.679900e+09,1649919600,NaN
1369195,veefriends-series-2,rodro-editions-2023,0.569274,NaN,NaN,34475.032761,1.673597e+09,1649919600,NaN
1366905,veefriends-series-2,messycatscollection,0.591736,NaN,NaN,34475.032761,1.665644e+09,1649919600,NaN
1369835,veefriends-series-2,space-passenger-1,0.592824,NaN,NaN,34475.032761,1.666076e+09,1649919600,NaN


In [84]:
der_lists = cfu.der_list_from_db()
result_filt = result_filt[~result_filt.apply(lambda row: (row['Top_100'], row['Alt']) in der_lists, axis=1)]

In [90]:
n_smallest = 1
def keep_knocks(group):
    try:
        return group.nlargest(n_smallest,'Top_ratio')
    except:
        return

# Keep the rows with the 100 smallest 'Euc_Distance' for each 'Top_100'
tops = result_filt.groupby('Top_100', group_keys=False).apply(keep_knocks)
tops.sort_values(by='Top_100_volume',ascending=False).query("Top_ratio>0.1")
tops.to_pickle("high_vol_ko.pkl")

In [75]:
command = "select distinct slug from overlap_counts"
rows = psql.execute_commands([command])
slugs = [row[0] for row in rows]

In [45]:
average_sale_price = result_df.groupby('Top_100')['sale_price'].mean()
for top_100, average_price in average_sale_price.items():
    if top_100 in slugs:
        print(result_df.query(f"Top_100=='{top_100}'"))
        try:
            print(f'Top_100: {top_100}, top_100 avg: {total_stats[top_100][2]} Average Sale Price: {average_price} Ratio:{average_price/total_stats[top_100][2]}')
        except:
            print('error')

     Top_100                   Alt  Distance  sale_price
0  0n1-force  neotokyopunkscollabs  0.473301    0.026584
1  0n1-force               vainego  0.477493    0.001253
2  0n1-force              mekazuki  0.495278    0.032092
3  0n1-force          roars-by-ntp  0.511114    0.031096
4  0n1-force    tokyo-brave-heroes  0.517561    0.076197
5  0n1-force             skullzuki  0.525995    0.036677
6  0n1-force          dreamzukinft  0.528295         NaN
7  0n1-force    tokyobraveheroes2g  0.529048    0.030837
8  0n1-force     0xvampire-project  0.534188    0.172809
9  0n1-force     ssa-international  0.542918    0.012790
Top_100: 0n1-force, top_100 avg: 1.6031270601110468 Average Sale Price: 0.046703941374839124 Ratio:0.02913302540823183
     Top_100                         Alt  Distance  sale_price
10  3landers                  frnds-club  0.335867    0.003387
11  3landers          the-pepe-chimpsons  0.339762    0.009003
12  3landers          toonsquad-official  0.361197    0.061089
13

In [81]:
average_sale_price = result_df.groupby('Top_100')['volume'].mean()
print(stats_df.query(f"Alt=='{top_100}'")['volume'].iloc(0)[0])
for top_100, average_price in average_sale_price.items():
    if top_100 in slugs:
        # print(result_df.query(f"Top_100=='{top_100}'"))
        try:
            print(f"""Top_100: {top_100}, top_100 avg: {stats_df.query(f"Alt=='{top_100}'")['volume'].iloc(0)[0]} Average Sale Price: {average_price} Ratio:{average_price/(stats_df.query(f"Alt=='{top_100}'")['volume'].iloc(0)[0])}""")
        except:
            print('error')

13481.059605394878
Top_100: 0n1-force, top_100 avg: 85523.39979944992 Average Sale Price: 348.87393794134135 Ratio:0.004079280509889005
Top_100: 3landers, top_100 avg: 36364.85627594372 Average Sale Price: 284.9046677143464 Ratio:0.007834615529687054
Top_100: adidasoriginals, top_100 avg: 49290.542643256915 Average Sale Price: 194.9944432743328 Ratio:0.003956021435706563
Top_100: alienfrensnft, top_100 avg: 31325.187138270732 Average Sale Price: 15792.898927296152 Ratio:0.5041597631192309
Top_100: artgobblers, top_100 avg: 49732.852255338585 Average Sale Price: 799.7236726526464 Ratio:0.016080390252839357
Top_100: azuki, top_100 avg: 722544.2127118501 Average Sale Price: 5562.99573491182 Ratio:0.00769917693207009
Top_100: azukielementals, top_100 avg: 31643.648999438075 Average Sale Price: 120944.53875691672 Ratio:3.8220793929001187
Top_100: beanzofficial, top_100 avg: 176801.51569079515 Average Sale Price: 848.6331079112334 Ratio:0.004799919868308097
Top_100: bored-ape-chemistry-club,

In [7]:
import counterfeit_utils as cfu
cfu.compute_all_intervals('hapeprime',10)

([('monsterapeclub-original',
   7048.882844986332,
   '2022-01-27',
   (7.000000000000001, 7.500000000000001)),
  ('theparrotboss',
   2.9267899999999996,
   '2022-06-29',
   (1.1500000000000001, 1.08)),
  ('lordsocietynft-v2', 1184.4843506710324, '2022-01-11', (nan, 6.79945)),
  ('aliencloaks', 0.01, '2023-09-23', (0.20450000000000002, 0.2368)),
  ('satoshi-apes',
   3.3586313899999958,
   '2022-06-17',
   (0.8500000000000001, 1.11)),
  ('official-lucky-elephant-club-jungleverse',
   0.17940000000000006,
   '2023-02-11',
   (0.37000000000000005, 0.33061)),
  ('frenzy-ducks-genesis',
   14.509900000000002,
   '2022-10-30',
   (0.5408000000000001, 0.45670000000000005)),
  ('mighty-llama-genesis', 30.990039902377966, '2022-10-22', (0.55, 0.537413)),
  ('queen-apiens-1',
   1.6302689989999986,
   '2023-04-24',
   (0.24999990000000002, 0.2949)),
  ('alterbase-by-smtc', 13.54259687688888, '2023-01-09', (0.44775, 0.4932)),
  ('dac-official',
   18.294800000000006,
   '2022-05-18',
   (0.757

In [100]:
result_df = vol_merge_df
result_df['Top_100_volume'] = result_df['Top_100'].map(top_100_map)
result_df['Alt_date'] = result_df['Alt'].map(alt_date_map)
result_df['Top_100_date'] = result_df['Top_100'].map(top_100_date_map)
result_df['Top_ratio'] = result_df['volume']/result_df['Top_100_volume']
result_filt = result_df.sort_values(by='Top_ratio',ascending=False)
result_filt
der_lists = cfu.der_list_from_db()
result_filt = result_filt[result_filt.apply(lambda row: (row['Top_100'], row['Alt']) in der_lists, axis=1)]
n_smallest = 1
def keep_knocks(group):
    try:
        return group.nlargest(n_smallest,'Top_ratio')
    except:
        return

# Keep the rows with the 100 smallest 'Euc_Distance' for each 'Top_100'
tops = result_filt.groupby('Top_100', group_keys=False).apply(keep_knocks)
tops = tops.sort_values(by='Top_100_volume',ascending=False).head(10)
tops.to_pickle("high_vol_der.pkl")

In [98]:
vol_merge_df.query("Top_100=='boredapeyachtclub' and Alt=='mutant-ape-yacht-club'")

,Top_100,Alt,Distance,sale_price,volume,Top_100_volume,Alt_date,Top_100_date,Top_ratio
171012,boredapeyachtclub,mutant-ape-yacht-club,0.525634,10.809283,1.102118e+06,1.460340e+06,NaN,1.619075e+09,0.7547
